<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

### Build dataset

I'm going to work with a single dataset for now — the GSM8K-XL math dataset for training the calculator tool. The dataset is provided by the original toolken authors in [this GitHub repo](https://github.com/Ber666/ToolkenGPT).

In [ ]:
path = '../model/tokenizer.model'
tokenizer = Tokenizer(path)

In [ ]:
with open('../data/gsm8k-xl/train.json', 'r') as f: data = json.load(f)
with open('../data/gsm8k-xl/func_dict.json', 'r') as f: func_dict = json.load(f)

In [ ]:
data[0]

{'text': "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Let's think step by step. Natalia sold 48/2 = 24 clips in May.\nNatalia sold 48+24 = 72 clips altogether in April and May.\n#### 72",
 'start_token_idx': [60, 80],
 'end_token_idx': [62, 82],
 'tar_eq': ['<divide>(48, 2)=24<eoe>', '<add>(48, 24)=72<eoe>'],
 'tar_number': ['24', '72']}

In [ ]:
func_dict

{'<add>': 0, '<subtract>': 1, '<multiply>': 2, '<divide>': 3}

In [1]:
#| echo: false
#| output: asis
show_doc(PromptDS)

---

### PromptDS

>      PromptDS (json)

Initialize self.  See help(type(self)) for accurate signature.

In [2]:
#| echo: false
#| output: asis
show_doc(DataLoaders)

---

### DataLoaders

>      DataLoaders (tds, vds, bs, **kwargs)

Initialize self.  See help(type(self)) for accurate signature.

In [ ]:
tds = PromptDS(data[:int(0.9*len(data))])
vds = PromptDS(data[int(0.9*len(data)):])
len(tds), len(vds)

(5448, 606)

In [ ]:
dls = DataLoaders(tds, vds, 1)
inp, label = next(iter(dls.train))
inp.shape, label.shape

(torch.Size([1, 112]), torch.Size([1, 112]))

The dataloader here is going to provide the model with an `input` and a `label`.

The `input`:

In [ ]:
a = tokenizer.decode(list([i.item() for i in inp[0]]))
print(a)

Juniper, the Irish Setter, has 4 bones. Her master gives her enough bones to double her number of bones. Unfortunately, the neighbor's dog steals away two of Juniper's bones.  How many bones does Juniper have remaining? Let's think step by step. Doubling 4 bones gives her 4*2=8 bones.
The neighbor's dog steals two bones, leaving 8-2=6 bones.
#### 6


The `label` is going to have the same structure as the input. However, the tokens in the input that represent an answer to a mathematical expression are going to be replaced by a new token that represents the operator required to calculate that result. Thus, the responsibility of the model is to, when faced with a mathematical expression, predict the operator token required to calculate that expression. Once the model is finetuned to do this, we'll do some additional processing in the inference method to formulate the arguments to be sent to an external calculator tool. In the case of other tools, the mechanism would be the same.

In [ ]:
m = torch.ne(inp, label)
m2 = torch.where(label == -100, True, False)
m3 = torch.where(label >= 32000, True, False)
l = torch.zeros_like(label)
l[~m] = label[~m]
lab = tokenizer.decode([i.item() for i in l[~m2]])
r = [a.item()-32000 for a in label[m3]]
for h in r: 
    op = [k for k,v in func_dict.items() if v == h]
    lab = lab.replace("⁇", op[0], 1)

In [ ]:
print(lab)

Juniper, the Irish Setter, has 4 bones. Her master gives her enough bones to double her number of bones. Unfortunately, the neighbor's dog steals away two of Juniper's bones.  How many bones does Juniper have remaining? Let's think step by step. Doubling 4 bones gives her 4*2= <multiply>  bones.
The neighbor's dog steals two bones, leaving 8-2= <subtract>  bones.
#### 6
